In [1]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import os
import pathlib
import shutil

c:\Users\91742\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
"""
Same dir structure as on Kaggle
input/
    lfw-dataset/
        csv files
        lfw-deepfunneled/
working/
    notebook
    data/
        train/
        val/
        test/
"""

'\nSame dir structure as on Kaggle\ninput/\n    lfw-dataset/\n        csv files\n        lfw-deepfunneled/\nworking/\n    notebook\n    data/\n        train/\n        val/\n        test/\n'

In [5]:
device = ("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
data_folder = '../input/lfw-dataset/'

In [8]:
lfw_allnames = pd.read_csv(data_folder+"lfw_allnames.csv")

image_paths = lfw_allnames.loc[lfw_allnames.index.repeat(lfw_allnames['images'])]
image_paths['image_path'] = 1 + image_paths.groupby('name').cumcount()
image_paths['image_path'] = image_paths.image_path.apply(lambda x: str(x).zfill(4))
image_paths['image_path'] = image_paths.name + "/" + image_paths.name + "_" + image_paths.image_path + ".jpg"
image_paths = image_paths.drop("images", axis=1)

In [86]:
num_ppl = 4

print(image_paths['name'].value_counts()[:num_ppl])
list_people = list(image_paths['name'].value_counts()[:num_ppl].keys())
list_num_images = list(image_paths['name'].value_counts()[:num_ppl])
print(list_people, list_num_images)

George_W_Bush      530
Colin_Powell       236
Tony_Blair         144
Donald_Rumsfeld    121
Name: name, dtype: int64
['George_W_Bush', 'Colin_Powell', 'Tony_Blair', 'Donald_Rumsfeld'] [530, 236, 144, 121]


In [87]:
num_for_each = 100
tmp_l = []
for name in list(image_paths['name'].value_counts()[:num_ppl].keys()):
    tmp_l.append(image_paths[image_paths.name==name].sample(num_for_each))
data = pd.concat(tmp_l)
print(data)

                 name                                image_path
1871    George_W_Bush      George_W_Bush/George_W_Bush_0351.jpg
1871    George_W_Bush      George_W_Bush/George_W_Bush_0172.jpg
1871    George_W_Bush      George_W_Bush/George_W_Bush_0419.jpg
1871    George_W_Bush      George_W_Bush/George_W_Bush_0127.jpg
1871    George_W_Bush      George_W_Bush/George_W_Bush_0530.jpg
...               ...                                       ...
1404  Donald_Rumsfeld  Donald_Rumsfeld/Donald_Rumsfeld_0034.jpg
1404  Donald_Rumsfeld  Donald_Rumsfeld/Donald_Rumsfeld_0028.jpg
1404  Donald_Rumsfeld  Donald_Rumsfeld/Donald_Rumsfeld_0079.jpg
1404  Donald_Rumsfeld  Donald_Rumsfeld/Donald_Rumsfeld_0049.jpg
1404  Donald_Rumsfeld  Donald_Rumsfeld/Donald_Rumsfeld_0020.jpg

[400 rows x 2 columns]


In [88]:
data_train, data_test = train_test_split(data, test_size=0.2)
data_train, data_val = train_test_split(data_train, test_size=0.2)

In [89]:
print(data_train.shape, data_val.shape, data_test.shape)

(256, 2) (64, 2) (80, 2)


In [90]:
data_root = './data/'

data_list = [data_train, data_val, data_test]
dirs = ['train', 'val', 'test']

# """             # (un)comment this line and run, to copy

# # remove data directory if it exists
if os.path.exists(data_root) and os.path.isdir(data_root):
    shutil.rmtree(data_root)

for i in range(len(dirs)):
    pathlib.Path(os.path.join(data_root, dirs[i])).mkdir(parents=True, exist_ok=True)
    
    for person in list_people:
        if len(data_train[data_train['name']==person])>0:
            pathlib.Path(os.path.join(data_root, dirs[i], person)).mkdir(parents=True, exist_ok=True)

    for im_path in data_list[i].image_path:
        name = data[data['image_path']==im_path]['name'].iloc[0]
        path_from = os.path.join(data_folder+'/lfw-deepfunneled/lfw-deepfunneled/', im_path)
        path_to = os.path.join(data_root, dirs[i], name)

        if not os.path.isfile(os.path.join(path_to, im_path)):
            shutil.copy(path_from, path_to)       # earlier (just copies image)

# """

In [91]:
train_path = os.path.join(data_root, dirs[0])
val_path = os.path.join(data_root, dirs[1])
test_path = os.path.join(data_root, dirs[2])

train_transform = transforms.Compose(transforms=[
    # transforms.RandomHorizontalFlip(),
    # transforms.Grayscale(num_output_channels=1),         # convert to grayscale
    transforms.ToTensor(),
    # transforms.Normalize(mean=0, std=255),      # output = (input-mean)/std
])
test_transform = transforms.Compose(transforms=[
    # transforms.Grayscale(num_output_channels=1),         # convert to grayscale
    transforms.ToTensor(),
    # transforms.Normalize(mean=0, std=255)
])

train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path, train_transform), shuffle=True     # batch_size
)
val_loader = DataLoader(
    torchvision.datasets.ImageFolder(val_path, test_transform), shuffle=True
)
test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path, test_transform), shuffle=True
)

In [94]:
for data in train_loader:
    print(data[0].shape, data[1].shape)
    print(data[0], data[1])
    print(torch.mean(data[0]))
    break
# Total train data is of shape (128, 3, 250, 250)

torch.Size([1, 3, 250, 250]) torch.Size([1])
tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0745, 0.0000, 0.0000],
          ...,
          [0.0314, 0.0000, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0039,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0039, 0.0039, 0.0039],
          [0.0000, 0.0000, 0.0000,  ..., 0.0039, 0.0039, 0.0039],
          [0.0000, 0.0000, 0.0000,  ..., 0.0824, 0.0039, 0.0039],
          ...,
          [0.0314, 0.0000, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0039,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0078, 0.0078, 0.0078,  ..., 0.0000, 0.0000, 0.0000],
          [0.0078, 0.0078, 0.0078,  ..., 0.0000, 0.0000, 0.0000

In [101]:
class FaceCNN(nn.Module):
    def __init__(self, num_input_channels, num_classes, stride=1, padding=1):
        super().__init__()

        self.network = nn.Sequential(

        nn.Conv2d(in_channels=num_input_channels, out_channels=50, kernel_size=3, stride=stride, padding=padding),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),

        nn.Conv2d(in_channels=50, out_channels=20, kernel_size=3, stride=stride, padding=padding),
        nn.ReLU(),

        nn.Flatten(),
        nn.Linear(in_features=20*125*125, out_features=num_classes)

        )

    def forward(self, input):
        output = self.network(input)
        return output

In [102]:
model = FaceCNN(num_input_channels=3, num_classes=len(list_people))
# for e in model.parameters():
#     print(e)

# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, weight_decay=1e-3)
loss_fn = nn.CrossEntropyLoss()
num_epochs = 10

In [103]:
def evaluate(loader, model):

    model.eval()

    score = 0
    cnt = 0

    with torch.no_grad():       # not training, so no need to calculate gradients
        for data in loader:
            # images, labels = data
            output = model(data[0])
            _, pred = torch.max(output.data, 1)
            score += float(torch.sum(pred==data[1].data))
            cnt += data[0].shape[0]

    return score/cnt

In [104]:
def train():
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        train_score = 0
        cnt = 0
        train_loss = 0

        model.train()
        
        for batch in train_loader:
            optimizer.zero_grad()
            
            output = model(batch[0])
            label = batch[1]
            
            # print(output, label)

            loss = loss_fn(output, label)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()

            _, pred = torch.max(output.data, 1)
            train_score += float(torch.sum(pred==label.data))
            cnt += batch[0].shape[0]

            # print(pred, label)

        train_acc = train_score/cnt
        val_acc = evaluate(val_loader, model)
        
        print("Epoch:", epoch, "\tLoss:", train_loss, "\tTraining Acc:", train_acc, "\tVal Acc:", val_acc)

        if val_acc > best_acc:
            torch.save(model.state_dict(),'best.model')
            best_acc = val_acc

In [ ]:
train()

### Train Stats

```
num_ppl=4, num_for_each=100, num_input_channels=3

Epoch: 0 	Loss: 357.3502341210842 	Training Acc: 0.29296875 	Val Acc: 0.28125
Epoch: 1 	Loss: 334.25504925101995 	Training Acc: 0.4453125 	Val Acc: 0.453125
Epoch: 2 	Loss: 291.24144549667835 	Training Acc: 0.55078125 	Val Acc: 0.4375
Epoch: 3 	Loss: 216.78546953666955 	Training Acc: 0.66015625 	Val Acc: 0.609375
Epoch: 4 	Loss: 154.33271293982398 	Training Acc: 0.7890625 	Val Acc: 0.609375
Epoch: 5 	Loss: 100.5769217426714 	Training Acc: 0.8828125 	Val Acc: 0.546875
Epoch: 6 	Loss: 71.6847092430553 	    Training Acc: 0.9140625 	Val Acc: 0.59375
Epoch: 7 	Loss: 54.84315806432892 	Training Acc: 0.9453125 	Val Acc: 0.578125
Epoch: 8 	Loss: 26.644126489559653 	Training Acc: 0.9765625 	Val Acc: 0.640625
```
